In [6]:
import requests
import ipykernel
import re
from notebook.notebookapp import list_running_servers
from notebook import notebookapp

from matplotlib import pyplot as plt
servers = list(notebookapp.list_running_servers())

TOKEN = servers[0]['token']

base_url = next(list_running_servers())['url']
r = requests.get(
    url=base_url + 'api/sessions',
    headers={'Authorization': 'token {}'.format(TOKEN),})

r.raise_for_status()
response = r.json()

kernel_id = re.search(
    'kernel-(.*).json',
    ipykernel.connect.get_connection_file()
).group(1)

NOTEBOOK_PATH = {
    r['kernel']['id']: r['notebook']['path']
    for r in response
}[kernel_id]
print(NOTEBOOK_PATH)

aposentadoria-ouro-comparacao/xgb.ipynb


In [14]:
import os
import glob
import pickle
from itertools import chain
from collections import Counter
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import fitz
from dodfminer.extract.polished.acts.aposentadoria import Retirements
import unidecode

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split, StratifiedKFold

from sklearn.model_selection import  cross_validate

from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import f1_score, accuracy_score
import xgboost as xgb

In [8]:
%%time
pipe = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('model', xgb.XGBClassifier(
        objective='multi:softprob',
        random_state=0,
        num_class=2
    ))
])

CPU times: user 147 µs, sys: 30 µs, total: 177 µs
Wall time: 182 µs


In [17]:
%%time
df = pd.read_csv('blocos_qtd_entidades.csv', index_col=False)
df['y'] = (df.qtd_ents > 0).astype(int)
df.head(2)

CPU times: user 1.09 s, sys: 95.8 ms, total: 1.18 s
Wall time: 1.18 s


,x0,x,y0,y1,text,qtd_ents,y
0,144.846329,190.436508,659.465149,208.861298,"ANO XLVII EDICAO No- 248\nBRASILIA - DF, TERCA...",0,0
1,56.702831,353.227753,344.850311,370.024231,"Secretaria de Estado de Trabalho, Desenvolvime...",0,0


In [12]:
# train_test_split(df.Text)

In [25]:
%%time

pipe = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('model', xgb.XGBClassifier(
        objective='multi:softprob',
        random_state=42,
        num_class=2,
        n_jobs=-1
    ))
])    

CPU times: user 188 µs, sys: 0 ns, total: 188 µs
Wall time: 194 µs


In [27]:
%%time
skf = StratifiedKFold(n_splits=5)
scores = {
    'acc': [],
    'f1': [],
}
for train_index, test_index in skf.split(df.text, df.y):
    x_train, x_test = df.text[train_index], df.text[test_index]
    y_train, y_test = df.y[train_index], df.y[test_index]
    pipe.fit(x_train, y_train)
    y_pred = pipe.predict(x_test)
    scores['acc'].append(f1_score(y_test, y_pred))
    scores['f1'].append(accuracy_score(y_test, y_pred))
#     break
# y_pred = pipe.predict(df_test.text)

CPU times: user 15min 49s, sys: 1.31 s, total: 15min 51s
Wall time: 15min 51s


In [28]:
scores

{'acc': [0.9686098654708519,
  0.9459459459459459,
  0.9732142857142857,
  0.9642857142857144,
  0.9866666666666667],
 'f1': [0.999661999034283,
  0.9994205417934231,
  0.9997102708967116,
  0.9996136945289488,
  0.9998551354483558]}

In [32]:
np.mean(scores['acc']), np.mean(scores['f1'])

(0.9677444956166928, 0.9996523283403445)

In [30]:
prf = 'pipe_xgb'
pickle.dump(pipe, open(f'{prf}.pkl', 'wb'))
open(f'{prf}.txt', 'w').write(
    f"notebook de origem: {NOTEBOOK_PATH}\n\n"
    f"`{prf}.pkl` contém um pipeline que contém uma instância\n"
    "treinada do modelo xgboost. A entrada é um vetor de strings\n"
    "e a saída um vetor de inteiros , 1-> verdadeiro 0-> falso\n"
    "(no que diz respeito à presença ou não de atos de aposentadoria).\n"
)

304